In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("pyspark_joins").getOrCreate()

In [2]:
# Sample data for DataFrame 1
data1 = [("Alice", 3000, "HR"),
         ("Bob", 4000, "Finance"),
         ("Charlie", 5000, "IT")]

# Sample data for DataFrame 2
data2 = [("Alice", "New York"),
         ("Bob", "Los Angeles"),
         ("David", "Chicago")]

# Creating DataFrames
df1 = spark.createDataFrame(data1, ["name", "salary", "department"])
df2 = spark.createDataFrame(data2, ["name", "city"])

In [3]:
df1.show()
df2.show()

+-------+------+----------+
|   name|salary|department|
+-------+------+----------+
|  Alice|  3000|        HR|
|    Bob|  4000|   Finance|
|Charlie|  5000|        IT|
+-------+------+----------+

+-----+-----------+
| name|       city|
+-----+-----------+
|Alice|   New York|
|  Bob|Los Angeles|
|David|    Chicago|
+-----+-----------+



inner join different ways

In [5]:
# Perform inner join on the 'name' column
df_inner = df1.join(df2, on="name", how="inner")
df_inner.show()

+-----+------+----------+-----------+
| name|salary|department|       city|
+-----+------+----------+-----------+
|Alice|  3000|        HR|   New York|
|  Bob|  4000|   Finance|Los Angeles|
+-----+------+----------+-----------+



In [6]:
# Perform inner join on the 'name' column
df_inner = df1.join(df2, df1.name==df2.name,'inner').show()

+-----+------+----------+-----+-----------+
| name|salary|department| name|       city|
+-----+------+----------+-----+-----------+
|Alice|  3000|        HR|Alice|   New York|
|  Bob|  4000|   Finance|  Bob|Los Angeles|
+-----+------+----------+-----+-----------+



left join different ways

In [7]:
# Perform left outer join on the 'name' column
df_left = df1.join(df2, on="name", how="left")

df_left.show()

+-------+------+----------+-----------+
|   name|salary|department|       city|
+-------+------+----------+-----------+
|  Alice|  3000|        HR|   New York|
|    Bob|  4000|   Finance|Los Angeles|
|Charlie|  5000|        IT|       null|
+-------+------+----------+-----------+



In [8]:
# Perform left outer join on the 'name' column
df_left = df1.join(df2,df1.name==df2.name,'left').show()

+-------+------+----------+-----+-----------+
|   name|salary|department| name|       city|
+-------+------+----------+-----+-----------+
|  Alice|  3000|        HR|Alice|   New York|
|    Bob|  4000|   Finance|  Bob|Los Angeles|
|Charlie|  5000|        IT| null|       null|
+-------+------+----------+-----+-----------+



full and right joins

In [9]:
df_right = df1.join(df2, df1.name==df2.name,'right')
df_right.show()

df_full = df1.join(df2,df1.name==df2.name,'full')
df_full.show()


+-----+------+----------+-----+-----------+
| name|salary|department| name|       city|
+-----+------+----------+-----+-----------+
|Alice|  3000|        HR|Alice|   New York|
|  Bob|  4000|   Finance|  Bob|Los Angeles|
| null|  null|      null|David|    Chicago|
+-----+------+----------+-----+-----------+

+-------+------+----------+-----+-----------+
|   name|salary|department| name|       city|
+-------+------+----------+-----+-----------+
|  Alice|  3000|        HR|Alice|   New York|
|    Bob|  4000|   Finance|  Bob|Los Angeles|
|Charlie|  5000|        IT| null|       null|
|   null|  null|      null|David|    Chicago|
+-------+------+----------+-----+-----------+



In [10]:
df_left_semi = df1.join(df2, df1.name==df2.name,'left_semi')
df_left_semi.show()


+-----+------+----------+
| name|salary|department|
+-----+------+----------+
|Alice|  3000|        HR|
|  Bob|  4000|   Finance|
+-----+------+----------+



In [11]:
df_left_anti = df1.join(df2, df1.name==df2.name,'left_anti')
df_left_anti.show()


+-------+------+----------+
|   name|salary|department|
+-------+------+----------+
|Charlie|  5000|        IT|
+-------+------+----------+



In [12]:
from pyspark.sql.functions import col

df_self_join = df1.alias("df1").join(
    df1.alias("df2"),
    col('df1.department') == col('df2.department'),
    'inner'
).select(
    col("df1.name").alias("name1"),
    col("df2.name").alias("name2"),
    col("df1.department")
)

df_self_join.show()


+-------+-------+----------+
|  name1|  name2|department|
+-------+-------+----------+
|    Bob|    Bob|   Finance|
|  Alice|  Alice|        HR|
|Charlie|Charlie|        IT|
+-------+-------+----------+



In [13]:
from pyspark.sql.functions import broadcast



# Sample large DataFrame
data_large = [("John", "HR"), ("Jane", "Finance"), ("Joe", "IT"), ("Jill", "HR")]
columns_large = ["name", "department"]
df_large = spark.createDataFrame(data_large, columns_large)

# Sample small DataFrame
data_small = [("HR", "Human Resources"), ("Finance", "Financial Department")]
columns_small = ["department", "department_name"]
df_small = spark.createDataFrame(data_small, columns_small)

df_large.show()

df_small.show()




+----+----------+
|name|department|
+----+----------+
|John|        HR|
|Jane|   Finance|
| Joe|        IT|
|Jill|        HR|
+----+----------+

+----------+--------------------+
|department|     department_name|
+----------+--------------------+
|        HR|     Human Resources|
|   Finance|Financial Department|
+----------+--------------------+



In [15]:
from pyspark.sql.functions import broadcast,col

df_broadcasted = df_large.join(
    broadcast(df_small),
    df_large["department"] == df_small["department"],
    "inner"
)
# Show the results
df_broadcasted.show()

+----+----------+----------+--------------------+
|name|department|department|     department_name|
+----+----------+----------+--------------------+
|John|        HR|        HR|     Human Resources|
|Jane|   Finance|   Finance|Financial Department|
|Jill|        HR|        HR|     Human Resources|
+----+----------+----------+--------------------+

